In [1]:
import torch

In [2]:
x = torch.FloatTensor([[1,2,3],[4,5,6]])
x

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [5]:
x_gpu = x.cuda()
x_gpu

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0')

In [6]:
x_cpu = x_gpu.cpu()
x_cpu

tensor([[1., 2., 3.],
        [4., 5., 6.]])

### 6) Tensor Size

In [7]:
# tensor.size() -> indexing also possible

x = torch.FloatTensor(10,12,3,3)

x.size()[:]

torch.Size([10, 12, 3, 3])

## 2. Indexing, Slicing, Joining
### 1) Indexing

In [8]:
# torch.index_select(input, dim, index)

x = torch.rand(4,3)
out = torch.index_select(x,0,torch.LongTensor([0,3]))

x,out

(tensor([[0.7714, 0.2144, 0.5901],
         [0.9377, 0.5185, 0.9854],
         [0.9385, 0.5839, 0.8058],
         [0.8013, 0.6593, 0.8861]]),
 tensor([[0.7714, 0.2144, 0.5901],
         [0.8013, 0.6593, 0.8861]]))

In [9]:
# pythonic indexing also works

x[:,0],x[0,:],x[0:2,0:2]

(tensor([0.7714, 0.9377, 0.9385, 0.8013]),
 tensor([0.7714, 0.2144, 0.5901]),
 tensor([[0.7714, 0.2144],
         [0.9377, 0.5185]]))

In [11]:
# torch.masked_select(input, mask)

x = torch.randn(2,3)
mask = torch.ByteTensor([[0,0,1],[0,1,0]])
out = torch.masked_select(x,mask)

x, mask, out

C:\Users\png\Anaconda3\envs\PyTorch\lib\site-packages\ipykernel_launcher.py:5: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:733.)
  """


(tensor([[ 1.2177, -0.6464, -1.0733],
         [ 0.8458, -0.4771, -0.0035]]),
 tensor([[0, 0, 1],
         [0, 1, 0]], dtype=torch.uint8),
 tensor([-1.0733, -0.4771]))

# Convolutional Neural Network
- MNIST data
- 3 convolutional layers
- 2 fully connected layers

## 1. Settings
### 1) Import required libraries

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable


### 3) Set hyperparameters

In [13]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

## 3. Data Generation

### 1) Download Data

In [14]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw




Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw
Processing...


C:\Users\png\Anaconda3\envs\PyTorch\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


### 2) Check Dataset

In [15]:
print(mnist_train.__getitem__(0), mnist_train.__len__())
mnist_test.__getitem__(0), mnist_test.__len__()

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

((tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
            0.0000, 0.0000, 0

### 3) Set DataLoader

In [16]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

## 4. Model & Optimizer

### 1) CNN Model

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer_1 = nn.Conv2d(1,16,5)
        self.act_1 = nn.ReLU()
        
        self.layer_2 = nn.Conv2d(16,32,5)
        self.act_2 = nn.ReLU()
        self.max_2 = nn.MaxPool2d(2,2)
        
        self.layer_3 = nn.Conv2d(32,64,5)
        self.act_3 = nn.ReLU()
        self.max_3 = nn.MaxPool2d(2,2)
        
        self.fc_layer_1 = nn.Linear(64*3*3,100)
        self.act_4 = nn.ReLU()
        self.fc_layer_2 = nn.Linear(100,10)
    
    def forward(self,x):
        out = self.layer_1(x)
        out = self.act_1(out)
        for module in list(self.modules())[2:-3]:
            out = module(out)
        out = out.view(batch_size,-1)
        for module in list(self.modules())[-3:]:
            out = module(out)

        return out

model = CNN().cuda()

### 2) Loss func & Optimizer

In [18]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 5. Train 

In [22]:
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = Variable(image).cuda()
        y_= Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)            

tensor(1.1175, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6270, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2764, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3839, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2967, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8085, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7348, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9758, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3381, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5008, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7621, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9380, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2562, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1841, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2672, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3318, device='cuda:0', grad_fn=<NllLossBackward

In [ ]:
#param_list = list(model.parameters())
#print(param_list)

## 6. Test

In [23]:
correct = 0
total = 0

for image,label in test_loader:
    x = Variable(image,volatile=True).cuda()
    y_= Variable(label).cuda()

    output = model.forward(x)
    _,output_index = torch.max(output,1)
        
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
print("Accuracy of Test Data: {}".format(100*correct/total))

C:\Users\png\Anaconda3\envs\PyTorch\lib\site-packages\ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Accuracy of Test Data: 94.29999542236328
